##### Copyright 2018 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=Afd8bu4xJOgh).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/swift/tutorials/custom_differentiation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/custom_differentiation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/custom_differentiation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Custom differentiation

This tutorial will show you how to define your own custom derivatives, perform derivative surgery, and implement your own gradient checkpointing API in just 5 lines of Swift.

## Declaring custom derivatives

You can define custom derivatives for any Swift function that has differentiable parameters and results. By doing that, you can even import a C function and make it differentiable.

In [0]:
import Glibc

func sillyExp(_ x: Float) -> Float {
    let 𝑒 = Float(M_E)
    print("Taking 𝑒(\(𝑒)) to the power of \(x)!")
    return pow(𝑒, x)
}

@differentiating(sillyExp)
func sillyDerivative(_ x: Float) -> (value: Float, pullback: (Float) -> Float) {
    let y = sillyExp(x)
    return (value: y, pullback: { v in v * y })
}

print("exp(3) =", sillyExp(3))
print("𝛁exp(3) =", gradient(of: sillyExp)(3))

Taking 𝑒(2.7182817) to the power of 3.0!
exp(3) = 20.085535
Taking 𝑒(2.7182817) to the power of 3.0!
𝛁exp(3) = 20.085535


## Stop derivatives from propagating

Commonly known as "stop gradient" in machine learning use cases, method [`withoutDerivative()`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE17withoutDerivativexyF) stops derivatives from propagating.

Plus, `withoutDerivative()` can sometimes help the Swift compiler with identifying what not to differentiate and producing more efficient derivaitves. When it is detectable that the derivative of a function will always be zero, the Swift compiler will produce a warning. Explicitly using `.withoutDerivative()` silences that warning.

In [0]:
let x: Float = 2.0
let y: Float = 3.0
gradient(at: x, y) { x, y in
    sin(sin(sin(x))) + cos(cos(cos(y))).withoutDerivative()
}

▿ 2 elements
  - .0 : -0.18009877
  - .1 : 0.0


## Derivative surgery

Method [`withGradient(_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE12withGradientyxy15CotangentVectorQzzcF) makes arbitrary operations (including mutation) run on the gradient at a value during the enclosing function’s backpropagation. 

Use this to debug or make experimental tweaks to backpropagation.

### It works anywhere

All differentiation APIs provided by the standard library are defined generically over all types that conform to the `Differentiable` protocol: `Float`, `Double`, `Float80`, SIMD vectors, and even your own types!

Read technical document [Differentiable Types](https://github.com/tensorflow/swift/blob/master/docs/DifferentiableTypes.md) for more insights on the `Differentiable` protocol.

In [0]:
var x: Float = 30
x.gradient { x -> Float in
    // Print the partial derivative with respect to the result of `sin(x)`.
    let a = sin(x).withGradient { print("∂+/∂sin = \($0)") } 
    // Force the partial derivative with respect to `x` to be `0.5`.
    let b = log(x.withGradient { (dx: inout Float) in
        print("∂log/∂x = \(dx), but rewritten to 0.5");
        dx = 0.5
    })
    return a + b
}

∂log/∂x = 0.033333335, but rewritten to 0.5
∂+/∂sin = 1.0


0.65425146


### Use it in a neural network module

Just like how we used it in a simple `Float` function, we can use it in any numerical application, like the following neural network built using the [Swift for TensorFlow Deep Learning Library](https://github.com/tensorflow/swift-apis).

In [0]:
import TensorFlow

struct MLP: Layer {
    var layer1 = Dense<Float>(inputSize: 2, outputSize: 10, activation: relu)
    var layer2 = Dense<Float>(inputSize: 10, outputSize: 1, activation: relu)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        let h0 = layer1.applied(to: input, in: context).withGradient { print("∂L/∂layer1 =", $0) }
        return layer2.applied(to: h0, in: context)
    }
}

let optimizer = SGD<MLP, Float>(learningRate: 0.02)
var classifier = MLP()
let context = Context(learningPhase: .training)

let x: Tensor<Float> = [[0, 0], [0, 1], [1, 0], [1, 1]]
let y: Tensor<Float> = [0, 1, 1, 0]

for _ in 0..<10 {
    let 𝛁model = classifier.gradient { classifier -> Tensor<Float> in
        let ŷ = classifier.applied(to: x, in: context).withGradient { print("∂L/∂ŷ =", $0) }
        let loss = (ŷ - y).squared().mean()
        print("Loss: \(loss)")
        return loss
    }
    optimizer.update(&classifier.allDifferentiableVariables, along: 𝛁model)
}

Loss: 0.33426732
∂L/∂ŷ = [[-0.25], [-0.078446716], [-0.12092987], [0.031454742]]
∂L/∂layer1 = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.03357383, -0.027463656, 0.037523113, -0.002631738, -0.030937709, -0.014981618, -0.02623924, -0.026290288, 0.027446445, 0.01046889], [-0.051755875, -0.042336714, 0.057843916, -0.004056967, -0.047692157, -0.023094976, -0.040449213, -0.040527906, 0.042310182, 0.016138362], [0.013462082, 0.0110120885, -0.015045625, 0.0010552468, 0.012405078, 0.006007172, 0.010521135, 0.010541604, -0.011005187, -0.0041977055]]
Loss: 0.33176333
∂L/∂ŷ = [[-0.24746439], [-0.07523262], [-0.11674469], [0.03514868]]
∂L/∂layer1 = [[-0.10602461, -0.08665162, 0.11829134, -0.008301959, -0.09804342, -0.04726032, -0.08280819, -0.082981184, 0.08658129, 0.032860693], [-0.032232955, -0.0263433, 0.035962217, -0.0025239112, -0.029806564, -0.014367796, -0.025174841, -0.025227433, 0.026321916, 0.009990108], [-0.050018553, -0.040879082, 0.05580555, -0.003916562, -0.046253316, -0

## Recomputing activations during backpropagation to save memory (checkpointing)

Checkpointing is a traditional technique in reverse-mode automatic differentiation to save memory when computing derivatives by making large intermediate values in the original computation not be saved in memory for backpropagation, but instead recomputed as needed during backpropagation. This technique has been realized in modern deep learning libraries as well. In Swift, API [`withComputationInPullbacks(_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE28withRecomputationInPullbacksyqd__qd__xcAaBRd__lF) makes you able to control what to recompute during backpropagation, and it is available on all `Differentiable` types.

But today, let us learn how to define our own gradient checkpointing APIs from scratch, in just a few lines of code.

### My gradient checkpointing API

We can define our own gradient checkpointing API, `makeRecomputedInGradient(_:)`, in terms of standard library function [`differentiableFunction(from:)`](https://www.tensorflow.org/swift/api_docs/Functions#/s:10TensorFlow22differentiableFunction4fromq0_x_q_tcq0_5value_15CotangentVectorQz_AEQy_tAEQy0_c8pullbacktx_q_tc_tAA14DifferentiableRzAaJR_AaJR0_r1_lF), which is a shorthand for creating a differentiable function directly from a derivative function (also called a "vector-Jacobian products (VJP) function").

As we have seen before, the derivative function returns a tuple of the original function's result and a pullback closure. We return `original(x)` in `value:`, and call `pullback(at:in:)` on `original` to evaluate the original function again and get a pullback.

In [0]:
/// Given a differentiable function, returns the same differentiable function except when
/// derivatives of this function is being computed, values in the original function that are needed
/// for computing the derivatives will be recomputed, instead of being captured by the differnetial
/// or pullback.
///
/// - Parameter body: The body of the differentiable function.
/// - Returns: The same differentiable function whose derivatives, when computed, will recompute
///   some values from the original function.
func makeRecomputedInGradient<T: Differentiable, U: Differentiable>(
    _ original: @escaping @differentiable (T) -> U
) -> @differentiable (T) -> U {
    return differentiableFunction { x in
        (value: original(x), pullback: { v in pullback(at: x, in: original)(v) })
    }
}

### Verify it works

In [0]:
let input: Float = 10.0
print("Running original computation...")

// Differentiable multiplication with checkpointing.
let square = makeRecomputedInGradient { (x: Float) -> Float in
    print("  Computing square...")
    return x * x
}

// Differentiate `f(x) = (cos(x))^2`.
let (output, backprop) = input.valueWithPullback { input -> Float in
    return square(cos(input))
}
print("Running backpropagation...")
let grad = backprop(1)
print("Gradient = \(grad)")

Running original computation...
  Computing square...
Running backpropagation...
  Computing square...
Gradient = -0.9129453


### Extend it to neural network modules

In this example, we define a simple convolutional neural network.

```swift
struct Model: Layer {
    var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6))
    var maxPool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var flatten = Flatten<Float>()
    var dense = Dense<Float>(inputSize: 36 * 6, outputSize: 10)

    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: conv, maxPool, flatten, dense)
    }
}
```

We want to make activations in the convolution layer (`conv`) be recomputed during backpropagation. However, using `makeRecomputedInGradient(_:)` could make the resulting code look cumbersome, especially when we want to apply layers sequentially using [`sequenced(in:through:_:_:_:_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE9sequenced2in7through____6OutputQyd_3_AA7ContextC_qd__qd_0_qd_1_qd_2_qd_3_t5InputQyd__RszAA5LayerRd__AaMRd_0_AaMRd_1_AaMRd_2_AaMRd_3_AKQyd_0_AGRtd__AKQyd_1_AGRtd_0_AKQyd_2_AGRtd_1_AKQyd_3_AGRtd_2_r3_lF).

```swift
input.sequenced(in: context, through: conv, maxPool, flatten, dense)
```

So, why don't we define a **special layer type** that wraps a layer and makes its activations be recomputed during backpropagation? Let's do it.

First, we define a `makeRecomputedInGradient(_:)` function that takes a binary function.

In [0]:
// Same as the previous `makeRecomputedInGradient(_:)`, except it's for binary functions.
func makeRecomputedInGradient<T: Differentiable, U: Differentiable, V: Differentiable>(
    _ original: @escaping @differentiable (T, U) -> V
) -> @differentiable (T, U) -> V {
    return differentiableFunction { x, y in
        (value: original(x, y), pullback: { v in pullback(at: x, y, in: original)(v) })
    }
}

Then, we define a generic layer `ActivationRecomputing<Wrapped>`.

In [0]:
/// A layer wrapper that makes the underlying layer's activations be discarded during application
/// and recomputed during backpropagation.
struct ActivationDiscarding<Wrapped: Layer>: Layer 
    where Wrapped.AllDifferentiableVariables == Wrapped.CotangentVector {
    /// The wrapped layer.
    var wrapped: Wrapped

    @differentiable
    func applied(to input: Wrapped.Input, in context: Context) -> Wrapped.Output {
        let apply = makeRecomputedInGradient { (layer: Wrapped, input: Input) -> Wrapped.Output in
            print("    Applying \(Wrapped.self) layer...")
            return layer.applied(to: input, in: context)
        }
        return apply(wrapped, input)
    }
}

Finally, we can add a method on all layers that returns the same layer except its activations are discarded during application and recomputeed during backpropagation.

In [0]:
extension Layer where AllDifferentiableVariables == CotangentVector {
    func discardingActivations() -> ActivationDiscarding<Self> {
        return ActivationDiscarding(wrapped: self)
    }
}

Back in the model, all we have to change is to wrap the convolution layer into the activation-recomputing layer.

```swift
var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6)).discardingActivations()
```

Now, simply use it in the model!

In [0]:
struct Model: Layer {
    var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6)).discardingActivations()
    var maxPool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var flatten = Flatten<Float>()
    var dense = Dense<Float>(inputSize: 36 * 6, outputSize: 10)

    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: conv, maxPool, flatten, dense)
    }
}

When we run a training loop, we can see that the convolution layer's activations are computed twice: once during layer application, and once during backpropagation.

In [0]:
// Use random training data.
let x = Tensor<Float>(randomNormal: [10, 16, 16, 3])
let y = Tensor<Int32>(rangeFrom: 0, to: 10, stride: 1)

var model = Model()
let opt = SGD<Model, Float>()
let context = Context(learningPhase: .training)

for i in 1...5 {
    print("Starting training step \(i)")
    print("  Running original computation...")
    let (logits, backprop) = model.appliedForBackpropagation(to: x, in: context)
    let (loss, dL_dŷ) = logits.valueWithGradient { logits in
        softmaxCrossEntropy(logits: logits, labels: y)
    }
    print("  Loss: \(loss)")
    print("  Running backpropagation...")
    let (dL_dθ, _) = backprop(dL_dŷ)
    
    opt.update(&model.allDifferentiableVariables, along: dL_dθ)
}

Starting training step 1
  Running original computation...
    Applying Conv2D<Float> layer...
  Loss: 3.6660562
  Running backpropagation...
    Applying Conv2D<Float> layer...
Starting training step 2
  Running original computation...
    Applying Conv2D<Float> layer...
  Loss: 3.1203392
  Running backpropagation...
    Applying Conv2D<Float> layer...
Starting training step 3
  Running original computation...
    Applying Conv2D<Float> layer...
  Loss: 2.7324893
  Running backpropagation...
    Applying Conv2D<Float> layer...
Starting training step 4
  Running original computation...
    Applying Conv2D<Float> layer...
  Loss: 2.4246051
  Running backpropagation...
    Applying Conv2D<Float> layer...
Starting training step 5
  Running original computation...
    Applying Conv2D<Float> layer...
  Loss: 2.1656146
  Running backpropagation...
    Applying Conv2D<Float> layer...


Just like that, it is super easy to define generic differentiable programming libraries for different domains.

## License

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.